## 记录
支持向量机kernel选择rbf。poly不行
(7000,100,10):0.49542035049422883,0.475699857506
(7000,100,100):0.47776052287345033,0.467821176851
(4000,100,10):0.52943281746425952,0.567164110138
(4000,100,100):0.51586121688791642,0.544996066217
(4000,100,50):0.51275929028512923,0.557419601773
(4000,10,20):0.5739563335296668,0.704022362125

In [292]:
# 引入这个项目需要的库
import numpy as np
import pandas as pd
#import visuals as vs
from IPython.display import display # 使得我们可以对DataFrame使用display()函数
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.decomposition import PCA
from sklearn.decomposition import RandomizedPCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.grid_search import GridSearchCV
import time

# 设置以内联的形式显示matplotlib绘制的图片（在notebook中显示更美观）
%matplotlib inline

# 载入整个数据集
try:
    data = pd.read_csv("first_train_index_20180131.csv")
    #data.drop(['Region', 'Channel'], axis = 1, inplace = True)
    print "dataset has {} samples with {} features each.".format(*data.shape)
except:
    print "Dataset could not be loaded. Is the dataset missing?"


dataset has 483851 samples with 2 features each.


In [293]:
#数据集中各类星体数量
data['type'].value_counts()

star       442969
unknown     34288
galaxy       5231
qso          1363
Name: type, dtype: int64

In [294]:
#抽样,4000数据集
data_star = data[data['type']== 'star'].sample(n=1000,random_state=0)
data_unknown = data[data['type']== 'unknown'].sample(n=1000,random_state=0)
data_galaxy = data[data['type']== 'galaxy'].sample(n=1000,random_state=0)
data_qso = data[data['type']== 'qso'].sample(n=1000,random_state=0)
data = pd.concat([data_star,data_unknown,data_galaxy,data_qso])

#设置index，提高检索效率
data = data.set_index('id', drop=True, append=False, inplace=False, verify_integrity=False)

In [295]:
#读取data_id对应波长文件

#四舍五入
import time
def ggg(x):
    x = float(x)
    x = x/10
    return round(x)*10

waves_counts = {}
count = 0

for i in data.index:
    f = open('/Users/suyangluojia/Desktop/LEARN/天池/天文数据挖掘/first_train_data_20180131/%s.txt'%i, 'r')
    waves_str = f.read()
    waves = waves_str.split(',')
    waves_float = map(ggg,waves)
    waves_counts[i] = pd.Series(waves_float).value_counts()
    f.close()
    print count,i
    count +=1

0 1089807
1 1348326
2 1176902
3 927833
4 1181221
5 719480
6 866392
7 702287
8 1123890
9 1333483
10 701287
11 934607
12 1027406
13 1077806
14 716513
15 1022735
16 1322236
17 912719
18 851516
19 861882
20 867034
21 980449
22 1088680
23 1309903
24 1221681
25 766490
26 883557
27 770081
28 979927
29 744819
30 1165796
31 1079527
32 873859
33 1268825
34 1008028
35 1146099
36 921801
37 1132073
38 1207302
39 1106124
40 824617
41 994538
42 1037808
43 860227
44 1175715
45 955667
46 1184362
47 1124091
48 1336677
49 743418
50 1021105
51 1296361
52 763789
53 836534
54 1364645
55 994092
56 1097919
57 721533
58 908657
59 1202387
60 1049429
61 1245371
62 1346191
63 1016111
64 1111998
65 945307
66 1223782
67 1294040
68 976450
69 1108909
70 982852
71 753295
72 1264646
73 1159991
74 1290592
75 1080128
76 863210
77 1011037
78 1111314
79 853742
80 1161380
81 758026
82 1197579
83 796684
84 1203302
85 741068
86 1051467
87 1187360
88 961384
89 1134781
90 739238
91 996305
92 981335
93 1330950
94 972057
95 76535

In [296]:
#各波段计数,约耗时10min
import time
t0 = time.time()
count = 0

for k in waves_counts.keys():
    for i in waves_counts[k].index:
        if i in data.columns:
            data.loc[k,i] = waves_counts[k][i]
            #print data.loc[k,i]
        else:
            data[i] = 0
            data.loc[k,i] = waves_counts[k][i]
    count +=1
    print count,k
    
print "耗时%0.3fs"%(time.time()-t0)

1 1187840
2 729089
3 1024003
4 786438
5 761863
6 1318920
7 1155084
8 909329
9 1002158
10 729110
11 1048599
12 1105945
13 961199
14 770081
15 722267
16 1073188
17 737318
18 1343527
19 1135322
20 1208327
21 1335340
22 1327150
23 1193425
24 1138740
25 1343542
26 1196087
27 753721
28 770106
29 1327167
30 860227
31 835652
32 1030838
33 1122376
34 1081417
35 776887
36 974924
37 720973
38 802897
39 853347
40 1118222
41 1130584
42 1269858
43 720996
44 1081446
45 942184
46 1359979
47 811116
48 761965
49 1114225
50 1155186
51 909429
52 794742
53 893050
54 1319036
55 1032213
56 1147009
57 816491
58 925828
59 1319045
60 1146782
61 1048711
62 999560
63 1193325
64 711967
65 868500
66 1103214
67 1138841
68 1089693
69 1040545
70 983067
71 942247
72 794792
73 1171627
74 737455
75 1056946
76 957129
77 753848
78 1237177
79 1245371
80 1360933
81 712893
82 1097919
83 1130688
84 1016003
85 745505
86 999624
87 1063628
88 1229002
89 1106124
90 975054
91 1056481
92 991443
93 991444
94 1327317
95 721110
96 7211

In [297]:
display(data.describe())

,10.0,0.0,20.0,30.0,-10.0,40.0,50.0,90.0,80.0,70.0,...,-4080.0,-28840.0,-5030.0,-3460.0,-16200.0,-64090.0,-20960.0,-2800.0,-4960.0,-81600.0
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,...,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,527.314500,369.091750,315.952750,196.201000,22.874000,133.332250,95.568000,39.705500,47.731500,56.576000,...,0.000250,0.000250,0.000250,0.000250,0.000250,0.000250,0.000250,0.000250,0.000250,0.000250
std,677.750106,653.818368,483.058094,366.255301,81.279707,288.116277,234.660006,123.893877,139.218937,157.151506,...,0.015811,0.015811,0.015811,0.015811,0.015811,0.015811,0.015811,0.015811,0.015811,0.015811
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,1.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,135.500000,29.000000,65.000000,26.000000,4.000000,10.000000,4.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,955.500000,366.000000,454.000000,210.500000,16.000000,108.000000,56.000000,8.000000,12.000000,19.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2516.000000,2600.000000,2388.000000,2227.000000,1568.000000,2031.000000,1991.000000,1490.000000,1412.000000,1567.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [298]:
#调整列顺序
cols = data.columns.sort_values()
data = data.ix[:, cols]

In [382]:
a = data.describe()

In [386]:
b = data.sum()

In [385]:
a.ix['mean'].sort_values()

-55296160.0      0.00025
 118480.0        0.00025
 118680.0        0.00025
 118720.0        0.00025
 118890.0        0.00025
 119070.0        0.00025
 119100.0        0.00025
 119240.0        0.00025
 119270.0        0.00025
 119290.0        0.00025
 119320.0        0.00025
 119350.0        0.00025
 119370.0        0.00025
 119440.0        0.00025
 119510.0        0.00025
 119550.0        0.00025
 119640.0        0.00025
 119680.0        0.00025
 119830.0        0.00025
 119890.0        0.00025
 119930.0        0.00025
 119940.0        0.00025
 118430.0        0.00025
 119950.0        0.00025
 118410.0        0.00025
 118350.0        0.00025
 117620.0        0.00025
 117630.0        0.00025
 117660.0        0.00025
 117670.0        0.00025
                 ...    
 270.0           6.74825
 260.0           6.91925
 250.0           7.26300
 240.0           7.67725
-20.0            8.30450
 230.0           8.49825
 220.0           9.39000
 210.0          10.06600
 200.0          10.62650


In [324]:
#训练数据集
data_y = data['type']
data_x = data.drop(['type'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.25, random_state=0)

In [325]:
#PCA
# TODO：通过在good_data上使用PCA，将其转换成和当前特征数一样多的维度
#pca = PCA(n_components=100)
#pca.fit(good_data)
# TODO：使用上面的PCA拟合将变换施加在log_samples上
#pca_samples = pca.transform(samples)
# 生成PCA的结果图
#pca_results = vs.pca_results(good_data, pca)

n_components = 20
print "Extracting the top %d eigenfaces from %d faces" % (n_components, X_train.shape[1])
t0 = time.time()
pca = PCA(n_components=n_components, whiten=True).fit(X_train)
X_train_pca = pca.transform(X_train)
print "done in %0.3fs" % (time.time() - t0)
print pca.explained_variance_ratio_

Extracting the top 20 eigenfaces from 17455 faces
done in 3.040s
[ 0.35891322  0.26088056  0.13836044  0.07787537  0.04702718  0.0305378
  0.02099755  0.0141531   0.00942957  0.0069709   0.00529578  0.00432761
  0.00377915  0.00299154  0.00225224  0.00180353  0.00150328  0.00141976
  0.00120365  0.00106944]


In [326]:
#将测试数据提取主成分
X_test_pca = pca.transform(X_test)

In [302]:
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score

print "Fitting the classifier to the training set"
t0 = time.time()
param_grid = {
         'C': [1e3, 5e3, 1e4, 5e4, 1e5, 1e6], 
         #'degree': [2,3,4],
         'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1, 1],
          }
# for sklearn version 0.16 or prior, the class_weight parameter value is 'auto'
#clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
clf = SVC(kernel='rbf', class_weight='balanced', C = 5000, gamma = 1)
clf = clf.fit(X_train_pca, y_train)
print "done in %0.3fs" % (time.time() - t0)
#print "Best estimator found by grid search:"
#print clf.best_estimator_

Fitting the classifier to the training set
done in 3.199s


In [303]:
from sklearn.metrics import f1_score
y_pred = clf.predict(X_test_pca)
results = accuracy_score(y_test,  y_pred)
f1_score = f1_score(y_test,  y_pred, average='macro')
results,f1_score

(0.58199999999999996, 0.5739563335296668)

In [338]:
from sklearn.grid_search import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, recall_score
from sklearn.metrics import f1_score

print "Fitting the classifier to the training set"
t0 = time.time()
param_grid = {
         'C': [1e3, 5e3, 1e4, 5e4, 1e5],
          'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
          }
# for sklearn version 0.16 or prior, the class_weight parameter value is 'auto'
clf_1 = MLPClassifier(solver='lbfgs', alpha=1e-4,hidden_layer_sizes=(15,), random_state=1)
clf_1 = clf_1.fit(X_train_pca, y_train)
print "done in %0.3fs" % (time.time() - t0)
print "Best estimator found by grid search:"
#print clf_1.best_estimator_

y_pred_1 = clf_1.predict(X_test_pca)
results_1 = accuracy_score(y_test,  y_pred_1)
f1_score_1 = f1_score(y_test,  y_pred_1, average='macro') 
print results_1,f1_score_1

Fitting the classifier to the training set
done in 0.406s
Best estimator found by grid search:
0.715 0.714220901161


In [339]:
data_test = pd.read_csv("first_test_index_20180131.csv")

In [342]:
data.columns

Index([-55296160.0, -40559920.0, -38301570.0, -36431440.0, -32370010.0,
       -30631800.0, -28159940.0, -26287530.0, -24781460.0, -24647790.0,
       ...
        63489740.0,  63748250.0,  66331550.0,  66625480.0,  68322840.0,
        69757680.0,  81770560.0,  84916960.0, 159068820.0,     u'type'],
      dtype='object', length=17456)

In [371]:
# 载入整个数据集
try:
    data_origin = pd.read_csv("first_train_index_20180131.csv")
    #data.drop(['Region', 'Channel'], axis = 1, inplace = True)
    print "dataset has {} samples with {} features each.".format(*data_origin.shape)
except:
    print "Dataset could not be loaded. Is the dataset missing?"


#抽样,4000数据集
data_star = data_origin[data_origin['type']== 'star'].sample(n=200,random_state=1)
data_unknown = data_origin[data_origin['type']== 'unknown'].sample(n=200,random_state=1)
data_galaxy = data_origin[data_origin['type']== 'galaxy'].sample(n=200,random_state=1)
data_qso = data_origin[data_origin['type']== 'qso'].sample(n=200,random_state=1)
data_test = pd.concat([data_star,data_unknown,data_galaxy,data_qso])

#设置index，提高检索效率
data_test = data_test.set_index('id', drop=True, append=False, inplace=False, verify_integrity=False)
#读取data_id对应波长文件

#四舍五入
import time
def ggg(x):
    x = float(x)
    x = x/10
    return round(x)*10

waves_counts = {}
count = 0

for i in data_test.index:
    f = open('/Users/suyangluojia/Desktop/LEARN/天池/天文数据挖掘/first_train_data_20180131/%s.txt'%i, 'r')
    waves_str = f.read()
    waves = waves_str.split(',')
    waves_float = map(ggg,waves)
    waves_counts[i] = pd.Series(waves_float).value_counts()
    f.close()
    print count,i
    count +=1
    
#各波段计数,约耗时10min
import time
t0 = time.time()
count = 0

for k in waves_counts.keys():
    for i in waves_counts[k].index:
        if i in data.columns:
            data_test[i] = 0
            data_test.loc[k,i] = waves_counts[k][i]
            #print data.loc[k,i]
        else:
            #data[i] = 0
            #data.loc[k,i] = waves_counts[k][i]
            continue
    count +=1
    print count,k
    
print "耗时%0.3fs"%(time.time()-t0)

dataset has 483851 samples with 2 features each.
0 1035748
1 920616
2 940601
3 1015792
4 1289708
5 745158
6 746261
7 1149386
8 1172165
9 1078829
10 1238918
11 1174054
12 782971
13 690502
14 1311125
15 852050
16 868529
17 1343805
18 855170
19 1170272
20 1251315
21 887618
22 921160
23 1279918
24 1138317
25 1096574
26 1248408
27 894284
28 804500
29 1120116
30 1320981
31 958566
32 718177
33 863853
34 735874
35 904488
36 902942
37 1267028
38 1075208
39 1358703
40 1050763
41 1241576
42 1347247
43 749750
44 777570
45 1081753
46 1241185
47 1303958
48 1361035
49 1066910
50 1243086
51 823896
52 1088945
53 924317
54 1290868
55 1158985
56 836337
57 700028
58 1161751
59 973745
60 1297640
61 1089025
62 999310
63 963650
64 702862
65 1232424
66 1151304
67 793587
68 1210565
69 861276
70 1142105
71 801507
72 1034484
73 946672
74 1108103
75 1117181
76 972773
77 726152
78 925478
79 1358902
80 842028
81 1014469
82 1232421
83 748589
84 926130
85 882839
86 1151587
87 1338127
88 912663
89 1081237
90 781963
91

In [372]:
for i in data.columns:
    if i in data_test.columns:
        continue
    else:
        data_test[i] = 0
#data_test = data_test.fillna(0)

,type,20.0,10.0,30.0,0.0,40.0,90.0,80.0,100.0,70.0,...,62960890.0,63489740.0,63748250.0,66331550.0,66625480.0,68322840.0,69757680.0,81770560.0,84916960.0,159068820.0
id,,,,,,,,,,,,,,,,,,,,,
1035748,star,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
920616,star,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
940601,star,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1015792,star,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1289708,star,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
745158,star,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
746261,star,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1149386,star,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1172165,star,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [376]:
display(data_test.describe())

,20.0,10.0,30.0,0.0,40.0,90.0,80.0,100.0,70.0,60.0,...,62960890.0,63489740.0,63748250.0,66331550.0,66625480.0,68322840.0,69757680.0,81770560.0,84916960.0,159068820.0
count,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,...,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0
mean,0.131250,0.026250,0.547500,0.002500,0.867500,0.143750,0.165000,0.080000,0.187500,0.416250,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,3.712311,0.742462,15.485639,0.070711,24.536605,4.065864,4.666905,2.262742,5.303301,11.773328,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,105.000000,21.000000,438.000000,2.000000,694.000000,115.000000,132.000000,64.000000,150.000000,333.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [377]:
y_test_origin = data_test['type']
X_test_origin = data_test.drop(['type'], axis = 1)

#将原始测试数据提取主成分
X_test_origin_pca = pca.transform(X_test_origin)

#原始测试数据预测
y_pred_origin = clf.predict(X_test_origin_pca)

#评分
results_ori = accuracy_score(y_pred_origin,y_test_origin)
f1_score_ori = f1_score(y_test_origin, y_pred_origin, average='macro') 
print results_ori,f1_score_ori

0.24875 0.0995995995996


In [379]:
pd.Series(y_pred_origin).value_counts()

star       799
unknown      1
dtype: int64